In [13]:
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from pymongo import GEOSPHERE
load_dotenv()

True

In [14]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("companies")

In [15]:
proyec = {"name":1,"category_code": 1, "_id": 0,"total_money_raised":1,  "offices.city":1, "offices.latitude":1, "offices.longitude":1}
cond = {"category_code": "games_video", "total_money_raised":{"$ne":"$0"}, "offices.latitude":{"$type": "double"},"offices.longitude":{"$type": "double"}}
list(companies.find(cond,proyec))[8]

{'name': 'Ustream',
 'category_code': 'games_video',
 'total_money_raised': '$60.1M',
 'offices': [{'city': 'San Francisco',
   'latitude': 37.392936,
   'longitude': -122.07948},
  {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
  {'city': 'Budapest', 'latitude': None, 'longitude': None}]}

In [16]:
selec = pd.DataFrame(list(companies.find(cond,proyec)))

In [18]:
selec.total_money_raised = selec.total_money_raised.str.strip()

In [19]:
selec["amount"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[1]
selec["mil"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[2]
selec["currency"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[0]

In [20]:
selec.currency.unique()

array(['$', '€', '£'], dtype=object)

In [21]:
selec.mil.unique()

array(['M', 'k'], dtype=object)

In [22]:
selec.head()

,name,category_code,total_money_raised,offices,amount,mil,currency
0,Lala,games_video,$44.2M,"[{'city': 'Palo Alto', 'latitude': 37.451151, ...",44.2,M,$
1,Joost,games_video,$45M,"[{'city': 'New York', 'latitude': 40.7464969, ...",45,M,$
2,Babelgum,games_video,$13.2M,"[{'city': 'London', 'latitude': 53.344104, 'lo...",13.2,M,$
3,Kyte,games_video,$23.4M,"[{'city': 'San Francisco', 'latitude': 37.7884...",23.4,M,$
4,Veoh,games_video,$69.8M,"[{'city': 'San Diego', 'latitude': 32.902266, ...",69.8,M,$


In [43]:
dol = selec[(selec["currency"] == '$') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
city1 = dol["offices"].values[0][0]["city"]

In [44]:
eur = selec[(selec["currency"] == '€') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
city2 = eur["offices"].values[0][0]["city"]

In [45]:
pounds = selec[(selec["currency"] == '£') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
city3 = pounds["offices"].values[0][0]["city"]

In [46]:
cities= [city1,city2,city3]

In [47]:
cities

['Austin', 'Hamburg', 'Newcastle Upon Tyne']